In [1]:
import tensorflow as tf
import keras
from tensorflow.keras.layers import Input, Dense, Conv2D
from tensorflow.keras import Sequential

from tensorflow.keras.models import Model
from tensorflow.keras.layers import UpSampling2D, MaxPooling2D, Flatten
import cv2
import os
import random
from tqdm import tqdm
import numpy as np
from keras.preprocessing import image
import os
import random
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import BatchNormalization
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
def get_data(folder_path):
#     temp = []
    temp = []
    features = []
    X = []
    count=0;
    try:
        for img in tqdm(os.listdir(folder_path)):
            if count>6499:
                break
            else:
                img = image.load_img(os.path.join(folder_path, img), target_size=(128, 128, 3))
                img_array = image.img_to_array(img)
                img_array /= 255.
        #             resize_array = cv2.resize(img_array, (128,128))
                temp.append(img_array)
                count+=1
                
    except:
        pass
#     for i in temp:
#         features.append(i)
#     X = np.array(temp).reshape(-1, 128, 128, 3)
#     X = X.astype('float32')
#     X /= 255
    return np.array(temp)

In [3]:
def get_data_validation(folder_path):
#     temp = []
    temp = []
    features = []
    X = []
    count=0;
    try:
        for img in tqdm(os.listdir(folder_path)):
            if count>6500 and count < 6913:
                img = image.load_img(os.path.join(folder_path, img), target_size=(128, 128, 3))
                img_array = image.img_to_array(img)
                img_array /= 255.
        #             resize_array = cv2.resize(img_array, (128,128))
                temp.append(img_array)
                count+=1
            else:
                count+=1
                
    except:
        pass
#     for i in temp:
#         features.append(i)
#     X = np.array(temp).reshape(-1, 128, 128, 3)
#     X = X.astype('float32')
#     X /= 255
    return np.array(temp)

In [ ]:
images_val = []
array_outwear_val = []
array_pants_val = []
array_shoes_val = []

In [ ]:
images = []
array_outwear = []
array_pants = []
array_shoes = []



In [ ]:
for file in os.listdir('/kaggle/input/re-polyvare/Re-PolyVore'):
    if file == 'outwear':
        
        
        array_outwear = get_data(os.path.join('/kaggle/input/re-polyvare/Re-PolyVore', file))
    elif file == 'pants':
        
        
        array_pants = get_data(os.path.join('/kaggle/input/re-polyvare/Re-PolyVore', file))
    elif file == 'shoes':
        
        
        array_shoes = get_data(os.path.join('/kaggle/input/re-polyvare/Re-PolyVore', file))

In [ ]:
for file in os.listdir('/kaggle/input/re-polyvare/Re-PolyVore'):
    if file == 'outwear':
        
        
        array_outwear_val = get_data_validation(os.path.join('/kaggle/input/re-polyvare/Re-PolyVore', file))
    elif file == 'pants':
        
        
        array_pants_val = get_data_validation(os.path.join('/kaggle/input/re-polyvare/Re-PolyVore', file))
    elif file == 'shoes':
        
        
        array_shoes_val = get_data_validation(os.path.join('/kaggle/input/re-polyvare/Re-PolyVore', file))

In [ ]:
len(array_outwear)

In [ ]:
len(array_outwear_val)

In [ ]:
model_outwear = keras.models.load_model('/kaggle/input/models-2/encoder_outwear.keras')
model_pants = keras.models.load_model('/kaggle/input/models-2/encoder_pants.keras')
model_shoes = keras.models.load_model('/kaggle/input/models-2/encoder_shoes.keras')

In [ ]:
columns=['Score_1','Score_2','Score_3','Score_4','Score_5','Score_6','Score_7','Score_8','Score_9','Score_10', 'Final_score']

In [ ]:
scores = pd.DataFrame(columns=columns)

In [ ]:
scores

In [ ]:
x = random.randint(50, 300)
x

In [ ]:
columns_new=['Score_1','Score_2','Score_3','Score_4','Score_5','Score_6','Score_7','Score_8','Score_9','Score_10']

In [ ]:
for data in columns_new:
    for i in range(6912):
        scores.loc[i, '{}'.format(data)] = random.randint(50, 300)

In [ ]:
scores['Final_score'] = scores.mean(axis=1)

In [ ]:
scores['Final_score']=scores['Final_score'].astype('int64')

In [ ]:
scores

In [ ]:
score = scores[['Final_score']]

In [ ]:
import seaborn as sns
sns.displot(score, kde=True)

In [ ]:
scaler = StandardScaler()

In [ ]:
score = scaler.fit_transform(score.to_numpy())

In [ ]:
score

In [ ]:
score = pd.DataFrame(score, columns=['Final_score'])

In [ ]:
score

In [ ]:
score_test = score[:6500]
score_val = score[6500:]

In [ ]:
score_test

In [ ]:
score_val

In [ ]:
outwear_input = Input(shape=(128,128,3), name='outwear_input')
pants_input = Input(shape=(128,128,3), name='pants_input')
shoes_input = Input(shape=(128,128,3), name='shoes_input')

# Connect the existing models to the new Input layers
outwear_layer = model_outwear(outwear_input)
pants_layer = model_pants(pants_input)
shoes_layer = model_shoes(shoes_input)

In [ ]:
combinedInput = concatenate([outwear_layer, pants_layer, shoes_layer])

In [ ]:
x = Flatten()(combinedInput)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Dense(1, activation='linear')(x)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model = Model(inputs=[outwear_input, pants_input, shoes_input], outputs=x)

In [ ]:
model.compile(loss='mean_absolute_percentage_error', optimizer='adam', metrics=[keras.metrics.MeanAbsolutePercentageError()])

In [ ]:
callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=5,
    verbose=1,
    mode="min",
    baseline=None,
    restore_best_weights=False
)

In [ ]:
model.summary()

In [ ]:
model.fit(x=[array_outwear,array_pants,array_shoes], y=score_test, epochs=200, validation_data = ([array_outwear_val,array_pants_val,array_shoes_val], score_val), callbacks=[callback])

In [ ]:
model.save('clothes.keras')

In [4]:
model=tf.keras.saving.load_model('/kaggle/input/wardrobe-2/clothes (1).keras')

In [15]:
def get_data_single(folder_path):
#     temp = []
    temp = []
    
    features = []
    X = []
    try:
#         for img in tqdm(os.listdir(folder_path)):
        img = image.load_img(os.path.join(folder_path), target_size=(128, 128, 3))
        img_array = image.img_to_array(img)
        img_array /= 255.
#             resize_array = cv2.resize(img_array, (128,128))
        temp.append(img_array)
    except:
        pass
#     for i in temp:
#         features.append(i)
#     X = np.array(temp).reshape(-1, 128, 128, 3)
#     X = X.astype('float32')
#     X /= 255
    return np.array(temp).reshape(-1, 128, 128, 3)

In [16]:
outwear_input=get_data_single('/kaggle/input/clothes/R.jpeg')
pants_input=get_data_single('/kaggle/input/clothes/pants.png')
shoes_input=get_data_single('/kaggle/input/clothes/shoes.jpeg')

In [17]:
outwear_input

array([[[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        ...,

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]]], dtype=float32)

In [18]:
zipped_input = tf.data.Dataset.zip(((outwear_input,pants_input,shoes_input), ))

TypeError: Invalid input to `zip`. Inputs are expected to be (nested) structures of `tf.data.Dataset` objects but encountered object of type <class 'numpy.ndarray'>.

In [21]:
y_pred = model.predict([outwear_input,pants_input,shoes_input])

1/1 [==============================] - 1s 829ms/step


In [22]:
pred = pd.DataFrame(columns=['Values'])

In [23]:
y_pred

array([[175.43973]], dtype=float32)

In [24]:
outwear = ['/kaggle/input/outwear-pics/R.jpeg','/kaggle/input/outwear-pics/outwear_2.jpeg', '/kaggle/input/outwear-pics/outwear_3.jpeg']
pants = ['/kaggle/input/pants-pics/pants.png','/kaggle/input/pants-pics/pants_2.webp', '/kaggle/input/pants-pics/pants_3.webp']
shoes = ['/kaggle/input/shoes-pics/shoes.jpeg','/kaggle/input/shoes-pics/shoes_2.jpeg', '/kaggle/input/shoes-pics/shoes_3.jpeg']

In [145]:
clothes = []

In [146]:
values=[]

In [147]:
outwear[0]

'/kaggle/input/outwear-pics/R.jpeg'

In [148]:
for i in range(3):
    for j in range(3):
        for k in range(3):
            if k==1:
                outwear_input=get_data_single(outwear[i])
            elif k==2 :
                pants_input=get_data_single(pants[j])
            elif k==3 :
                shoes_input=get_data_single(shoes[k])
            y_pred = model.predict([outwear_input,pants_input,shoes_input])
            values.append(y_pred[0][0])
            clothes.append([outwear[i], pants[j], shoes[k]])

1/1 [==============================] - 0s 326ms/step


In [149]:
len(values)

27

In [150]:
for value in range(len(values)):
    values[value] = round(values[value], 2)

In [151]:
values 

[168.31,
 175.44,
 175.44,
 175.44,
 175.44,
 175.44,
 175.44,
 175.44,
 175.44,
 175.44,
 174.63,
 174.64,
 174.64,
 174.64,
 174.63,
 174.63,
 174.63,
 174.63,
 174.63,
 168.31,
 168.32,
 168.32,
 168.32,
 168.31,
 168.31,
 168.31,
 168.31]

In [152]:
len(clothes[0])

3

In [153]:
clothes[0]

['/kaggle/input/outwear-pics/R.jpeg',
 '/kaggle/input/pants-pics/pants.png',
 '/kaggle/input/shoes-pics/shoes.jpeg']

In [154]:
temp_2 = []

In [155]:
for i in range(len(values)):
    temp_2.append((i,values[i]))

In [156]:
temp_2

[(0, 168.31),
 (1, 175.44),
 (2, 175.44),
 (3, 175.44),
 (4, 175.44),
 (5, 175.44),
 (6, 175.44),
 (7, 175.44),
 (8, 175.44),
 (9, 175.44),
 (10, 174.63),
 (11, 174.64),
 (12, 174.64),
 (13, 174.64),
 (14, 174.63),
 (15, 174.63),
 (16, 174.63),
 (17, 174.63),
 (18, 174.63),
 (19, 168.31),
 (20, 168.32),
 (21, 168.32),
 (22, 168.32),
 (23, 168.31),
 (24, 168.31),
 (25, 168.31),
 (26, 168.31)]

In [157]:
from operator import itemgetter

In [175]:
temp_2 = sorted(temp_2,key=itemgetter(1), reverse=True)

In [176]:
temp_2

[(1, 175.44),
 (3, 175.44),
 (4, 175.44),
 (5, 175.44),
 (6, 175.44),
 (7, 175.44),
 (8, 175.44),
 (9, 175.44),
 (11, 174.64),
 (12, 174.64),
 (13, 174.64),
 (10, 174.63),
 (14, 174.63),
 (15, 174.63),
 (16, 174.63),
 (17, 174.63),
 (18, 174.63),
 (20, 168.32),
 (21, 168.32),
 (22, 168.32),
 (0, 168.31),
 (19, 168.31),
 (23, 168.31),
 (24, 168.31),
 (25, 168.31),
 (26, 168.31)]

In [181]:
len(temp_2)

26

In [182]:
seen = set()
new = []
for i in range(len(temp_2)):
    if temp_2[i][1] not in seen:
        seen.add(temp_2[i][1])
        new.append(temp_2[i])

In [185]:
new

[(1, 175.44), (11, 174.64), (10, 174.63), (20, 168.32), (0, 168.31)]

In [186]:
seen


{168.31, 168.32, 174.63, 174.64, 175.44}

In [172]:
len(temp_2)

26

In [173]:
temp_2

[(1, 175.44),
 (3, 175.44),
 (4, 175.44),
 (5, 175.44),
 (6, 175.44),
 (7, 175.44),
 (8, 175.44),
 (9, 175.44),
 (11, 174.64),
 (12, 174.64),
 (13, 174.64),
 (10, 174.63),
 (14, 174.63),
 (15, 174.63),
 (16, 174.63),
 (17, 174.63),
 (18, 174.63),
 (20, 168.32),
 (21, 168.32),
 (22, 168.32),
 (0, 168.31),
 (19, 168.31),
 (23, 168.31),
 (24, 168.31),
 (25, 168.31),
 (26, 168.31)]

In [187]:
import matplotlib.pyplot as plt
import matplotlib.image as img 
import cv2
for i in range(len(new)):
    index = new[i][0]
    for j in clothes[index]:
        print(j)
    print(new[i][1])
#         array = img.imread(j)
#         plt.imshow(array)
    print('-------------------------------------------------------------------------------------------------')

/kaggle/input/outwear-pics/R.jpeg
/kaggle/input/pants-pics/pants.png
/kaggle/input/shoes-pics/shoes_2.jpeg
175.44
-------------------------------------------------------------------------------------------------
/kaggle/input/outwear-pics/outwear_2.jpeg
/kaggle/input/pants-pics/pants.png
/kaggle/input/shoes-pics/shoes_3.jpeg
174.64
-------------------------------------------------------------------------------------------------
/kaggle/input/outwear-pics/outwear_2.jpeg
/kaggle/input/pants-pics/pants.png
/kaggle/input/shoes-pics/shoes_2.jpeg
174.63
-------------------------------------------------------------------------------------------------
/kaggle/input/outwear-pics/outwear_3.jpeg
/kaggle/input/pants-pics/pants.png
/kaggle/input/shoes-pics/shoes_3.jpeg
168.32
-------------------------------------------------------------------------------------------------
/kaggle/input/outwear-pics/R.jpeg
/kaggle/input/pants-pics/pants.png
/kaggle/input/shoes-pics/shoes.jpeg
168.31
----------------

In [ ]:
for j in clothes[index]:
        print(j)
        array = img.imread(j)
        plt.imshow(array)

In [ ]:

array = img.imread('/kaggle/input/pants-pics/pants.png')
plt.imshow(array)

In [ ]:
array = img.imread('/kaggle/input/outwear-pics/R.jpeg')
plt.imshow(array)